In [ ]:
"""libraries required"""
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install timm
# !pip install torchattacks

In [ ]:
"""connect to google drive to store and retrieve data"""
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Dataset Loader**

In [ ]:
# import h5py
# import torch
# import numpy as np
# from torch.utils.data import Dataset
# import torchvision

# transform = torchvision.transforms.Resize((224, 224), antialias=True)
# device = "cuda" if torch.cuda.is_available() else "cpu"

# data_list = []
# label_list = []

# labels = {'airport': 0, 'shopping_mall': 1, 'metro_station': 2, 'street_pedestrian': 3, 'public_square': 4,
#           'street_traffic': 5, 'tram': 6, 'bus': 7, 'metro': 8, 'park': 9}

# # Open the HDF5 file
# h5_file_path = '/content/drive/MyDrive/data/datasets/DCASE19/DCASE19.hdf5'
# output_h5_file_path = '/content/drive/MyDrive/data/AdvData/DCASE19.h5'

# with h5py.File(h5_file_path, 'r') as hf:
#     with h5py.File(output_h5_file_path, 'w') as hf_out:
#         # Iterate over the samples in the HDF5 file
#         i = 0
#         for s, r in hf.items():
#             label_name = s.split('-')[0]
#             label_number = labels.get(label_name)

#             tensor_data = torch.tensor(r[:]).float() / 255.0
#             tensor_data = tensor_data.unsqueeze(0)
#             tensor_data = transform(tensor_data)

#             grp = hf_out.create_group(f'sample_{i}')
#             grp.create_dataset('data', data=tensor_data.cpu().numpy())
#             grp.create_dataset('label', data=label_number)
#             i += 1

# print(f'dataset stored at {output_h5_file_path}')

# Load directly from h5

In [ ]:
from torch.utils.data import TensorDataset
import h5py
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# data_list = []
# label_list = []

# Open the HDF5 file
# h5_file_path = '/content/drive/MyDrive/data/AdvData/DCASE19.h5'
# with h5py.File(h5_file_path, 'r') as hf:
#     # Iterate over the samples in the HDF5 file
#     for sample_name in hf.keys():
#         data = torch.tensor(hf[sample_name]['data'][:])
#         label = hf[sample_name]['label'][()]

#         # Append data and label to lists
#         data_list.append(data)
#         label_list.append(label)

# dataset = TensorDataset(torch.stack(data_list), torch.tensor(label_list))

In [ ]:
len(dataset)

13859

**Function to train the model and calculate metrics**

In [ ]:
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import random_split
import torch.nn.functional as F
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, accuracy_score
from sklearn.manifold import TSNE
import seaborn as sns

# function to train the model
def train_model(model, dataset, learning_rate=2e-4, num_epochs=10, batch_size=128, stepSize=10, gamma=0.1, validation_split=0.2, patience=3):
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()
    scheduler = StepLR(optimizer, step_size=stepSize, gamma=gamma)

    # Split dataset into training and validation sets
    num_samples = len(dataset)
    num_validation_samples = int(validation_split * num_samples)
    num_training_samples = num_samples - num_validation_samples
    training_dataset, validation_dataset = random_split(dataset, [num_training_samples, num_validation_samples])

    train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

    best_valid_loss = float('inf')
    patience_counter = 0
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        model.train()
        for batch_input, batch_target in train_loader:
            batch_input, batch_target = batch_input.to(device), batch_target.to(device)
            optimizer.zero_grad()

            predictions = model(batch_input)

            loss = loss_fn(predictions, batch_target)
            loss.backward()

            optimizer.step()

        print(f"  Training Loss: {loss.item()}")

        # Validation phase
        model.eval()
        with torch.no_grad():
            valid_loss = 0
            correct = 0
            total = 0
            for batch_input, batch_target in valid_loader:
                batch_input, batch_target = batch_input.to(device), batch_target.to(device)
                predictions = model(batch_input)
                _, predicted = torch.max(predictions, 1)
                total += batch_target.size(0)
                correct += (predicted == batch_target).sum().item()
                valid_loss += loss_fn(predictions, batch_target).item()

            accuracy = correct / total
            valid_loss /= len(valid_loader)
            print(f"  Validation Loss: {valid_loss}, Accuracy: {accuracy * 100:.2f}%")

            # Check for early stopping
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Early stopping after {patience} epochs of no improvement.")
                    return

        scheduler.step()

    print("Training finished.")

# function to calculate metrics
def calculate_metrics(model, dataset):
    data_loader = DataLoader(dataset, batch_size=16, shuffle=True)
    model = model.to(device)
    model.eval()
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for batch_input, batch_target in data_loader:
            batch_input, batch_target = batch_input.to(device), batch_target.to(device)
            predictions = model(batch_input)
            all_predictions.append(predictions.argmax(dim=1))
            all_targets.append(batch_target)

    all_predictions = torch.cat(all_predictions).cpu().numpy()
    all_targets = torch.cat(all_targets).cpu().numpy()

    accuracy = accuracy_score(all_targets, all_predictions)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    # precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_predictions, average='weighted')
    # print(f"Precision: {precision:.4f}")
    # print(f"Recall: {recall:.4f}")
    # print(f"F1 Score: {f1:.4f}")

    # cm = confusion_matrix(all_targets, all_predictions)
    # # Plot the confusion matrix using Seaborn
    # sns.heatmap(cm, annot=True)
    # plt.title("Confusion Matrix")
    # plt.xlabel("Predicted Label")
    # plt.ylabel("True Label")
    # plt.show()

In [ ]:
import torchattacks
import h5py
from torch.utils.data import TensorDataset
import time

def generate_adversarial(dataset, model, model_name):
    data_loader = DataLoader(dataset, batch_size=32)
    model = model.to(device)

    attack_dict = {}
    attack_dict['fgsm'] = torchattacks.FGSM(model, eps=8/255)
    attack_dict['bim'] = torchattacks.BIM(model, eps=8/255, alpha=2/255, steps=10)
    attack_dict['pgd'] = torchattacks.PGD(model, eps=8/255, alpha=1/255, steps=10, random_start=True)
    attack_dict['cw'] = torchattacks.CW(model, c=1, kappa=0, steps=50, lr=0.01)

    for name, attk in attack_dict.items():
        st = time.time()
        i = 0
        current_batch = 0
        h5_file_path = f'/content/drive/MyDrive/data/AdvData/DCASE19/{model_name}/{name}.h5'
        with h5py.File(h5_file_path, 'a') as hf:
            print(f"started {name}")
            for batch_input, batch_target in data_loader:

                # generate adv image for image
                batch_input, batch_target = batch_input.to(device), batch_target.to(device)
                adv_images = attk(batch_input, batch_target)

                # write in h5 file
                for j in range(batch_input.shape[0]):
                    grp_name = f'sample_{i}'
                    if grp_name in hf:
                        del hf[grp_name]  # Delete existing group
                    grp = hf.create_group(f'sample_{i}')
                    grp.create_dataset('data', data=adv_images[j].cpu().numpy())
                    grp.create_dataset('label', data=batch_target[j].cpu())
                    i += 1
                if (current_batch % 100 == 0):
                    print(f"{current_batch} - ", end="")
                current_batch += 1
            print("\n")

        end = time.time()
        print(f'{name} adv dataset for {model_name} stored, time {end-st}')

"""
runs the given model on all adv dataset of `adv_dir` directory
"""
def calculate_accuracy_for_adv_data(model, model_name, adv_dir):
    attk_list = ['fgsm', 'bim', 'pgd', 'cw']

    for attk in attk_list:
        h5_file_path = f'/content/drive/MyDrive/data/AdvData/DCASE19/{adv_dir}/{attk}.h5'
        data_list = []
        label_list = []

        with h5py.File(h5_file_path, 'r') as hf:
            # Iterate over the samples in the HDF5 file
            for sample_name in hf.keys():
                data = torch.tensor(hf[sample_name]['data'][:])
                label = hf[sample_name]['label'][()]

                # Append data and label to lists
                data_list.append(data)
                label_list.append(label)

        dataset_class = TensorDataset(torch.stack(data_list), torch.tensor(label_list))
        print(f'Stats for {model_name} on {adv_dir}/{attk} dataset', end=" ")
        calculate_metrics(model, dataset_class)
        del dataset_class
        del data_list
        del label_list


def for_all(model, model_name):
    adv_dirs = ['resnet50', 'resnet18', 'vit_base', 'mixer']
    for adv in adv_dirs:
        calculate_accuracy_for_adv_data(model, model_name, adv)

# ResNet 18

In [ ]:
import timm

resnet18 = timm.create_model('resnet18.a1_in1k', pretrained=True)
resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet18.fc = nn.Linear(in_features=512, out_features=10, bias=True)

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()
train_model(resnet18, dcase, learning_rate=1e-4, num_epochs=12, batch_size=128)

torch.save(resnet18.state_dict(), '/content/drive/MyDrive/data/Models/DCASE19/resnet18.pth')

In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/resnet18.pth')
resnet18.load_state_dict(checkpoint)

# calculate_metrics(resnet18, dataset)

Accuracy: 80.98%


# Resnet 50

In [ ]:
import timm

resnet50 = timm.create_model('resnet50.a1_in1k', pretrained=True)
resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet50.fc = nn.Linear(in_features=2048, out_features=10, bias=True)

In [ ]:
torch.cuda.empty_cache()
train_model(resnet50, dcase, learning_rate=1e-4, num_epochs=12, batch_size=64, stepSize=8)

torch.save(resnet50.state_dict(), '/content/drive/MyDrive/data/Models/DCASE19/resnet50.pth')

In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/resnet50.pth')
resnet50.load_state_dict(checkpoint)
for_all(resnet50, 'resnet50')

Stats for resnet50 on resnet50/fgsm dataset Accuracy: 8.83%
Stats for resnet50 on resnet50/bim dataset Accuracy: 0.00%
Stats for resnet50 on resnet50/pgd dataset Accuracy: 0.00%
Stats for resnet50 on resnet50/cw dataset Accuracy: 0.07%
Stats for resnet50 on resnet18/fgsm dataset Accuracy: 28.67%
Stats for resnet50 on resnet18/bim dataset Accuracy: 48.25%
Stats for resnet50 on resnet18/pgd dataset Accuracy: 51.49%
Stats for resnet50 on resnet18/cw dataset Accuracy: 78.60%
Stats for resnet50 on vit_base/fgsm dataset Accuracy: 33.71%
Stats for resnet50 on vit_base/bim dataset Accuracy: 49.15%
Stats for resnet50 on vit_base/pgd dataset Accuracy: 54.20%
Stats for resnet50 on vit_base/cw dataset Accuracy: 78.87%
Stats for resnet50 on mixer/fgsm dataset Accuracy: 36.42%
Stats for resnet50 on mixer/bim dataset Accuracy: 53.71%
Stats for resnet50 on mixer/pgd dataset Accuracy: 53.19%
Stats for resnet50 on mixer/cw dataset Accuracy: 79.41%


# VIT Base

In [ ]:
import timm

vit_base = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k', pretrained=True)
vit_base.patch_embed.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
vit_base.head = nn.Linear(in_features=768, out_features=10, bias=True)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()
train_model(vit_base, dcase, learning_rate=1e-5, num_epochs=12, batch_size=32, patience=3)

torch.save(vit_base.state_dict(), '/content/drive/MyDrive/data/Models/DCASE19/vit_base.pth')

Epoch 1/12
  Training Loss: 1.529644250869751
  Validation Loss: 1.6833921122825009, Accuracy: 33.45%
Epoch 2/12
  Training Loss: 1.712351679801941
  Validation Loss: 1.5002490158738762, Accuracy: 41.25%
Epoch 3/12
  Training Loss: 1.1851561069488525
  Validation Loss: 1.3651209453056599, Accuracy: 47.96%
Epoch 4/12
  Training Loss: 1.138542652130127
  Validation Loss: 1.3282636835657318, Accuracy: 50.49%
Epoch 5/12
  Training Loss: 0.9792410135269165
  Validation Loss: 1.2010698544568028, Accuracy: 53.59%
Epoch 6/12
  Training Loss: 1.0341508388519287
  Validation Loss: 1.124776055072916, Accuracy: 58.14%
Epoch 7/12
  Training Loss: 0.899638295173645
  Validation Loss: 1.1451198575140416, Accuracy: 57.42%
Epoch 8/12
  Training Loss: 0.5624861121177673
  Validation Loss: 0.9694701556501717, Accuracy: 63.84%
Epoch 9/12
  Training Loss: 0.2540462613105774
  Validation Loss: 0.9296337739489544, Accuracy: 66.55%
Epoch 10/12
  Training Loss: 0.379729688167572
  Validation Loss: 0.9062014814

In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/vit_base.pth')
vit_base.load_state_dict(checkpoint)

for_all(vit_base, 'vit_base')

Stats for vit_base on resnet50/fgsm dataset Accuracy: 80.93%
Stats for vit_base on resnet50/bim dataset Accuracy: 87.22%
Stats for vit_base on resnet50/pgd dataset Accuracy: 87.65%
Stats for vit_base on resnet50/cw dataset Accuracy: 90.04%
Stats for vit_base on resnet18/fgsm dataset Accuracy: 79.65%
Stats for vit_base on resnet18/bim dataset Accuracy: 86.53%
Stats for vit_base on resnet18/pgd dataset Accuracy: 87.31%
Stats for vit_base on resnet18/cw dataset Accuracy: 89.96%
Stats for vit_base on vit_base/fgsm dataset Accuracy: 5.19%
Stats for vit_base on vit_base/bim dataset Accuracy: 0.00%
Stats for vit_base on vit_base/pgd dataset Accuracy: 0.01%
Stats for vit_base on vit_base/cw dataset 

KeyboardInterrupt: 

# VIT Large

In [ ]:
import timm

vit_large = timm.create_model('vit_large_patch16_224.augreg_in21k_ft_in1k', pretrained=True)
vit_large.patch_embed.proj = nn.Conv2d(1, 1024, kernel_size=(16, 16), stride=(16, 16))
vit_large.head = nn.Linear(in_features=1024, out_features=10, bias=True)

In [ ]:
torch.cuda.empty_cache()
train_model(vit_large, dcase, learning_rate=1e-5, num_epochs=12, batch_size=16)

torch.save(vit_large.state_dict(), '/content/drive/MyDrive/data/Models/DCASE19/vit_large.pth')

In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/vit_large.pth')
vit_base.load_state_dict(checkpoint)

calculate_metrics(vit_large, dataset)

In [ ]:
for_all(vit_base, 'vit_base')

# mixer

In [ ]:
import timm
mixer = timm.create_model('mixer_b16_224.goog_in21k_ft_in1k', pretrained=True)
mixer.stem.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
mixer.head = nn.Linear(in_features=768, out_features=10, bias=True)

model.safetensors:   0%|          | 0.00/240M [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()
train_model(mixer, dcase, learning_rate=1e-4, num_epochs=12, batch_size=64)

torch.save(mixer.state_dict(), '/content/drive/MyDrive/data/Models/DCASE19/mixer.pth')

Epoch 1/12
  Training Loss: 1.988483190536499
  Validation Loss: 1.7201913757757707, Accuracy: 37.89%
Epoch 2/12
  Training Loss: 1.606507420539856
  Validation Loss: 1.2947275394743138, Accuracy: 49.01%
Epoch 3/12
  Training Loss: 0.8739612102508545
  Validation Loss: 0.9987240609797564, Accuracy: 61.64%
Epoch 4/12
  Training Loss: 0.6760417819023132
  Validation Loss: 0.8281339555978775, Accuracy: 68.03%
Epoch 5/12
  Training Loss: 0.8595892786979675
  Validation Loss: 0.7224954177032817, Accuracy: 72.46%
Epoch 6/12
  Training Loss: 0.2867172062397003
  Validation Loss: 0.6901143409989097, Accuracy: 75.24%
Epoch 7/12
  Training Loss: 0.021618274971842766
  Validation Loss: 0.6682053411548788, Accuracy: 78.53%
Epoch 8/12
  Training Loss: 0.02592005580663681
  Validation Loss: 0.7515407946299423, Accuracy: 77.63%
Epoch 9/12
  Training Loss: 0.19544252753257751
  Validation Loss: 0.8770877454768528, Accuracy: 76.25%
Epoch 10/12
  Training Loss: 0.10138049721717834
  Validation Loss: 0.8

In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/mixer.pth')
mixer.load_state_dict(checkpoint)

calculate_metrics(mixer, dataset)

Accuracy: 92.56%


In [ ]:
for_all(mixer, 'mixer')

Stats for mixer on resnet50/fgsm dataset Accuracy: 2.96%
Stats for mixer on resnet50/bim dataset Accuracy: 3.21%
Stats for mixer on resnet50/pgd dataset Accuracy: 3.06%
Stats for mixer on resnet50/cw dataset Accuracy: 3.06%
Stats for mixer on resnet18/fgsm dataset Accuracy: 3.01%
Stats for mixer on resnet18/bim dataset Accuracy: 3.11%
Stats for mixer on resnet18/pgd dataset Accuracy: 2.96%
Stats for mixer on resnet18/cw dataset Accuracy: 3.16%
Stats for mixer on vit_base/fgsm dataset Accuracy: 3.21%
Stats for mixer on vit_base/bim dataset Accuracy: 3.06%
Stats for mixer on vit_base/pgd dataset Accuracy: 3.06%
Stats for mixer on vit_base/cw dataset Accuracy: 3.11%
Stats for mixer on mixer/fgsm dataset Accuracy: 3.06%
Stats for mixer on mixer/bim dataset Accuracy: 3.01%
Stats for mixer on mixer/pgd dataset Accuracy: 3.16%
Stats for mixer on mixer/cw dataset Accuracy: 3.16%
